In [35]:
import os
import cv2
import time
import torch
import imutils
import numpy as np
import torch.nn.functional as F

from torch import nn, optim
from tqdm.notebook import tqdm
from torchvision import models, datasets, transforms

In [44]:
use_cuda = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
# Function to test custom images
from PIL import Image

def with_or_without_mask(image, model):
    input_img = Image.fromarray(image)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    input_tensor = preprocess(input_img)
    input_batch = input_tensor.unsqueeze(0)
    
    # Move the input to gpu if available
    
    input_batch = input_batch.to(use_cuda)


        
    with torch.no_grad():
        output = model(input_batch)
        
    prob, index = torch.max(F.softmax(output[0]), 0)
    
    # predicted class index and probability
    return prob, index

In [54]:
print("Loading face detector model...")
prototxtPath = 'D:\\UET\HK1_Year3\\CV\\face_detector\\deploy.prototxt'
weightsPath = 'D:\\UET\\HK1_Year3\\CV\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("Loading face mask detector model")


model = models.resnet50(pretrained = True)
in_ft = model.fc.in_features
model.fc = nn.Linear(in_features=in_ft, out_features=2, bias = False)

model.load_state_dict(torch.load('D:\\UET\\HK1_Year3\\CV\\model\\mask_detection_model.pt', map_location=torch.device('cpu')))
model.eval()
# cap = cv2.VideoCapture('https://www.youtube.com/watch?v=uDacvzuR0bA')
cap = cv2.VideoCapture(0) # Using device camera
_, image = cap.read()
# print(type(image))
# orig = image.copy()
# (h, w) = image.shape[:2]


# # construct a blob from the image
# blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

# # pass the blob through the network and obtain the face detections
# print("Computing face detections...")
# net.setInput(blob)
# detections = net.forward()
# for i in range(0, detections.shape[2]):
#     # extract the confidence (i.e, probability) associated with the detection
#     confidence = detections[0, 0, i, 2]
    
#     # filter out weak detections by ensuring the confidence is greater
#     # than the minimum confidence
#     if confidence > 0.5:
#         # compute the (x, y)-coordinates of the bounding box for the object
#         box = detections[0, 0, i, 3:7]  * np.array([w, h, w, h])
#         (startX, startY, endX, endY) = box.astype("int")
        
#         # ensure the bounding boxes fall within the dimensions of the frame
#         (startX, startY) = (max(0, startX), max(0, startY))
#         (endX, endY) = (min(w-1, endX), min(h-1, endY))
        
#         # preprocess the image and get the class and proba
#         prob, class_ = with_or_without_mask(image[startY:endY, startX:endX], model)
        
#         # determine the class label and color we'll use to draw the bounding
#         # box and text
#         label = "Mask" if class_ == 0 else "No Mask"
#         color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
#         # include the probability in the label
#         label = "{}: {:.2f}%".format(label, prob * 100)
        
#         # display the label and bounding box rectangle on the output frame
#         cv2.putText(image, label, (startX, startY - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
#         cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

# # show the output image
# cv2.imshow("output", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Loading face detector model...
Loading face mask detector model
<class 'NoneType'>


In [61]:
use_cuda

device(type='cpu')

In [63]:
def detect_and_predict(frame, faceNet, maskNet):
    # grab the dimension of the frame and then construct a blob from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))
    
    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    # initialize our list of faces, their corresponding locations
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence associated with the detections
        confidence = detections[0, 0, i, 2]
        
        # filter out weak detections by ensuring the confidence greater than
        # the minimum confidence
        if confidence > 0.7:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
            
            # ensure the bounding boxex fall within the dimension of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            # extract the face ROI and preprocess and predict
            face = frame[startY:endY, startX:endX]
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
    # only make a predictions if atleast one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on all the
        # faces at the same time rather than one-by-one prediction
        # in the above for loop
        for face in faces:
            pred = with_or_without_mask(face, maskNet)
            preds.append(pred)
    return (locs, preds)

In [64]:
print("Loading face detector model...")
prototxtPath = 'D:\\UET\HK1_Year3\\CV\\face_detector\\deploy.prototxt'
weightsPath = 'D:\\UET\\HK1_Year3\\CV\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("Loading face mask detector model")


model = models.resnet50(pretrained = True)
in_ft = model.fc.in_features
model.fc = nn.Linear(in_features=in_ft, out_features=2, bias = False)

model.load_state_dict(torch.load('D:\\UET\\HK1_Year3\\CV\\model\\mask_detection_model.pt', map_location=torch.device('cpu')))
model.eval()
# Starting video stream
# vs = cv2.VideoCapture('http://192.168.225.24:8080/video')
vs = cv2.VideoCapture(0)
# time.sleep(2.0)

# loop over the frames from the video stream
while True:
    
    # grab the frame from the threaded video stram and resize to have a
    # minimum of 400 px
    _, frame = vs.read()
#     frame = imutils.resize(frame, width=400)
    
    # detect faces in the frame and determine if they are wearing
    # a face mask or not
    (locs, preds) = detect_and_predict(frame, faceNet, model)
    
    # loop over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (prob, clas) = pred
        
        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if clas == 0 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, prob * 100)
        
        # display the label and bounding box rectangle on the original frame
        cv2.putText(frame, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    
    # show the output frame
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.release()

Loading face detector model...
Loading face mask detector model


d:\UET\HK1_Year3\CV\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\UET\HK1_Year3\CV\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3136\2771452379.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob, index = torch.max(F.softmax(output[0]), 0)
